In [189]:
!pip install num2words
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import numpy as np
import math
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [190]:
df = pd.read_csv('covid_tweets.csv')
democrat_tweets_df = df[df.party == 'Democrat']
republican_tweets_df = df[df.party == 'Republican']
democrat_tweets_df = democrat_tweets_df.sample(frac = 0.5)
frames = [democrat_tweets_df, republican_tweets_df]
df = pd.concat(frames)

In [191]:
df.head()

,name,twitter,party,tweet,tweet_published
34724,Greg Stanton,RepGregStanton,Democrat,"Thank you, @dougducey, this is the right thing...",2020-04-06
14973,Lucille Roybal-Allard,RepRoybalAllard,Democrat,As we all confront the threat of the #coronavi...,2020-03-13
34430,Ann Kirkpatrick,RepKirkpatrick,Democrat,I understood the challenges that local newspap...,2020-07-28
21239,Alan S. Lowenthal,RepLowenthal,Democrat,"If you have any #COVID19 symptoms, stay home. ...",2020-03-15
16774,Terri A. Sewell,RepTerriSewell,Democrat,Please take a moment to sign up for free text ...,2020-03-16


In [192]:
import tensorflow as tf
tf.test.is_gpu_available( cuda_only=False, min_cuda_compute_capability=None )
print(tf.__version__)

2.3.0


In [193]:
from nltk.tokenize import word_tokenize
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
covid_stop_words = {"covid", "covid19", "covid 19", "corona", "coronavirus"}
porter = PorterStemmer()
def preprocess_and_tokenize(tweet):
    # Remove non-letters, lowercase everything, remove stop words, and stem
    lower_letters = re.sub(r'[^A-Za-z0-9 ]+', " ", tweet).lower().split()
    important_words = []
    for w in lower_letters:
        if w not in stop_words and w not in covid_stop_words:
            important_words.append(w)
    return important_words

In [194]:
df['tokenize_tweet'] = df['tweet'].apply(lambda x : preprocess_and_tokenize(x))

In [195]:
dems, reps = list(), list()
for l in df.party:
    if l == 'Democrat':
        dems.append(1)
        reps.append(0)
    elif l == 'Republican':
        reps.append(1)
        dems.append(0)
df['dems']= dems
df['reps']= reps

In [196]:
df.head()

,name,twitter,party,tweet,tweet_published,tokenize_tweet,dems,reps
34724,Greg Stanton,RepGregStanton,Democrat,"Thank you, @dougducey, this is the right thing...",2020-04-06,"[thank, dougducey, right, thing, small, busine...",1,0
14973,Lucille Roybal-Allard,RepRoybalAllard,Democrat,As we all confront the threat of the #coronavi...,2020-03-13,"[confront, threat, please, read, newsletter, d...",1,0
34430,Ann Kirkpatrick,RepKirkpatrick,Democrat,I understood the challenges that local newspap...,2020-07-28,"[understood, challenges, local, newspapers, fa...",1,0
21239,Alan S. Lowenthal,RepLowenthal,Democrat,"If you have any #COVID19 symptoms, stay home. ...",2020-03-15,"[symptoms, stay, home, voted, ensure, american...",1,0
16774,Terri A. Sewell,RepTerriSewell,Democrat,Please take a moment to sign up for free text ...,2020-03-16,"[please, take, moment, sign, free, text, updat...",1,0


In [197]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(df, test_size=0.25, random_state=42)

### We build training vocab/max trailing sentence length/total number of words

In [198]:
all_training_words = [word for tokens in data_train["tokenize_tweet"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokenize_tweet"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

694371 words total, with a vocabulary size of 32817
Max sentence length is 91


### Build testing vocabulary 

In [199]:
all_test_words = [word for tokens in data_test["tokenize_tweet"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokenize_tweet"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

232213 words total, with a vocabulary size of 19069
Max sentence length is 77


In [43]:
### Load up word2vec
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
word2vec.wv.save_word2vec_format('googlenews.txt')

C:\Users\quynh\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


In [126]:
# Tokenize & Pad the Sequences

In [200]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["tweet"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["tweet"].tolist())
train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))
train_rnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 36680 unique tokens.


Now we will get embeddings from Google News Word2Vec model and save them corresponding to the sequence number we assigned to each word. If we could not get embeddings we save a random vector for that word.

In [201]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(36681, 300)


In [202]:
test_sequences = tokenizer.texts_to_sequences(data_test["tweet"].tolist())
test_rnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

### Develop Bi-LSTM

In [203]:
label_names = ['dems', 'reps']

In [204]:
y_train = data_train[label_names].values
x_train = train_rnn_data
y_tr = y_train

In [205]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU')) 
print(tf.__version__)

False
[]
2.3.0


In [206]:
def recurrent_nn(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    lstm = LSTM(256)(embedded_sequences)
    
    x = Dense(128, activation='relu')(lstm)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [207]:
import tensorflow as tf
# tf.test.is_gpu_available( cuda_only=False, min_cuda_compute_capability=None )
print(tf.__version__)

2.3.0


In [208]:
rnn_model = recurrent_nn(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM,  len(list(label_names)))

Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 50)]              0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 50, 300)           11004300  
_________________________________________________________________
lstm_8 (LSTM)                (None, 256)               570368    
_________________________________________________________________
dense_20 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 258       
Total params: 11,607,822
Trainable params: 603,522
Non-trainable params: 11,004,300
___________________________________

### Train Bi-LSTM

In [209]:
num_epochs = 5
batch_size = 20

In [210]:
hist = rnn_model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.10, shuffle=True, batch_size=batch_size)

Epoch 1/5
1440/1440 [==============================] - 117s 81ms/step - loss: 0.6275 - acc: 0.6444 - val_loss: 0.5817 - val_acc: 0.6987
Epoch 2/5
1440/1440 [==============================] - 111s 77ms/step - loss: 0.5470 - acc: 0.7148 - val_loss: 0.5155 - val_acc: 0.7324
Epoch 3/5
1440/1440 [==============================] - 110s 77ms/step - loss: 0.5006 - acc: 0.7465 - val_loss: 0.4901 - val_acc: 0.7599
Epoch 4/5
1440/1440 [==============================] - 114s 79ms/step - loss: 0.4554 - acc: 0.7776 - val_loss: 0.4824 - val_acc: 0.7702
Epoch 5/5
1440/1440 [==============================] - 114s 79ms/step - loss: 0.3980 - acc: 0.8097 - val_loss: 0.5091 - val_acc: 0.7702


In [211]:
predictions = rnn_model.predict(test_rnn_data, batch_size=512, verbose=1)

21/21 [==============================] - 5s 243ms/step


In [212]:
labels = [1, 0]
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])
sum(data_test['dems'] == prediction_labels)/len(prediction_labels)

0.7585592345933777

In [213]:
# Creating a CNN
# 1. Embeddings matrix passed to embedding layer
# 2. Five different filter sizese are applied to each tweet
# 3. GlobalMaxPooling1D layers are applied to each layer
# 4. All outputs concatenated
# 5. Dropout -> Dense -> Dropout -> Dense applied
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model

In [214]:
def cnn(embeddings, max_seq_length, num_words, 
        embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                               embedding_dim,
                               weights=[embeddings],
                               input_length=max_seq_length,
                               trainable=False)
    
    sequence_input = Input(shape=(max_seq_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    
    convs = []
    filter_sizes = [2,3,4,5,6]
    
    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, 
                        kernel_size=filter_size, 
                        activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)
        
    l_merge = concatenate(convs, axis=1)
    
    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [215]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300
cnn_model = cnn(train_embedding_weights, 
                MAX_SEQUENCE_LENGTH, 
                len(train_word_index)+1, 
                EMBEDDING_DIM, 
                len(list(label_names)))

Model: "functional_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 50, 300)      11004300    input_14[0][0]                   
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 49, 200)      120200      embedding_14[0][0]               
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 48, 200)      180200      embedding_14[0][0]               
______________________________________________________________________________________

In [216]:
num_epochs = 5
batch_size = 20

In [217]:
hist = cnn_model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.10, shuffle=True, batch_size=batch_size)

Epoch 1/5
1440/1440 [==============================] - 49s 34ms/step - loss: 0.6433 - acc: 0.6128 - val_loss: 0.5361 - val_acc: 0.7284
Epoch 2/5
1440/1440 [==============================] - 48s 33ms/step - loss: 0.5344 - acc: 0.7274 - val_loss: 0.5173 - val_acc: 0.7543
Epoch 3/5
1440/1440 [==============================] - 50s 35ms/step - loss: 0.4441 - acc: 0.7972 - val_loss: 0.4473 - val_acc: 0.7837
Epoch 4/5
1440/1440 [==============================] - 49s 34ms/step - loss: 0.3426 - acc: 0.8539 - val_loss: 0.4538 - val_acc: 0.7934
Epoch 5/5
1440/1440 [==============================] - 48s 33ms/step - loss: 0.2418 - acc: 0.9034 - val_loss: 0.4919 - val_acc: 0.7962


In [218]:
predictions = cnn_model.predict(test_rnn_data, batch_size=1024, verbose=1)

11/11 [==============================] - 4s 348ms/step


In [219]:
labels = [1, 0]
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])
sum(data_test['dems'] == prediction_labels)/len(prediction_labels)

0.7779757996435607